In [ ]:
"""
Created on Thu Mar 30 11:42 2023

This script is to check the reference melt for all ice shelves

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

FIXED MASK FILE FOR OPM016

In [ ]:
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_OPM016/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
home_path = '/bettik/burgardc/'
plot_path='/bettik/burgardc/PLOTS/NN_plots/input_vars/'

In [ ]:
ref_Gt_list = []
ref_box1_list = []
for nemo_run in ['bf663','bi646']:
    ref_Gt_time_list = []
    ref_box1_time_list = []
    for yy in range(1980, 1980 + 60):
        inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
        NEMO_melt_rates_1D = xr.open_dataset(inputpath_data+'melt_rates_1D_NEMO_oneFRIS_'+str(yy)+'.nc')
        NEMO_box1_myr = xr.open_dataset(inputpath_data+'melt_rates_box1_NEMO_oneFRIS_'+str(yy)+'.nc')
        ref_Gt_time_list.append(NEMO_melt_rates_1D['melt_Gt_per_y_tot'].assign_coords({'time': yy}))
        ref_box1_time_list.append(NEMO_box1_myr['mean_melt_box1_myr'].assign_coords({'time': yy}))
    ref_Gt = xr.concat(ref_Gt_time_list, dim='time') 
    ref_box1 = xr.concat(ref_box1_time_list, dim='time') 
    
    ref_Gt_list.append(ref_Gt.assign_coords({'nemo_run': nemo_run}))
    ref_box1_list.append(ref_box1.assign_coords({'nemo_run': nemo_run}))
        


In [ ]:
run_list = ['OPM006','OPM016','OPM018','OPM021']
for n,nemo_run in enumerate(run_list):
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot']
    ref_Gt_list.append(ref_Gt.assign_coords({'nemo_run': nemo_run}))
    NEMO_box1_myr = xr.open_dataset(outputpath_melt+'melt_rates_box1_NEMO_oneFRIS.nc')
    ref_box1 = NEMO_box1_myr['mean_melt_box1_myr']
    ref_box1_list.append(ref_box1.assign_coords({'nemo_run': nemo_run}))

In [ ]:
ref_Gt_all = xr.concat(ref_Gt_list, dim='nemo_run')
ref_box1_all = xr.concat(ref_box1_list, dim='nemo_run')

In [ ]:
ref_Gt_all = ref_Gt_all.assign_coords({'time': range(1,len(ref_Gt_all.time)+1)})
ref_box1_all = ref_box1_all.assign_coords({'time': range(1,len(ref_box1_all.time)+1)})

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,29,13,58,71,45,30,31,61,73,47,32,48,33,17,49,34,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,26,42,55] #62


for kisf in tqdm(sorted_isf_all):
    
        
    ax[i] = f.add_subplot(7,7,i+1)
    
    for n,nrun in enumerate(ref_Gt_all.nemo_run):
        
        if nrun == 'bf663':
            ccolor = 'orange'
        elif nrun == 'bi646':
            ccolor = 'red'
        else:
            ccolor = 'grey'
        
        ax[i].plot(ref_Gt_all.time, ref_Gt_all.sel(nemo_run=nrun, Nisf=kisf), color=ccolor)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    ax[i].set_xlim(0,60)
    ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'timeseries_melt_all.pdf')